In [ ]:
import pandas as pd
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, A3,A2,landscape
from reportlab.lib.units import inch,cm
from reportlab.lib import colors
import itertools
import math

In [ ]:
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [ ]:
def writeInRange(dframe,row,col1,col2,val):
    for i in range(col1,col2+1):
        dframe.iloc[row,i]=dframe.iloc[row,i]+val
    return dframe



def buildAllocationMatrix(df,rframe):
    data={'Code':list(rframe['Code']),'Resource_name':list(rframe['Name']),'Unit':rframe['Unit']}    
    df2=pd.DataFrame(data)
    start=min(list(df['Start']))
    finish=max(list(df['Finish']))
    daytext=dayTextGenerator(start,finish)
    zerolist=[0 for x in list(rframe['Code']) ]
    k=3
    for text in daytext:
        df2.insert(k,text,zerolist,True)
        k=k+1
    return df2
def calcualteResourceAllocation(cframe,aframe):
    start=min(list(cframe['Start']))
    finish=max(list(cframe['Finish']))
    daytext=dayTextGenerator(start,finish)
    """preparing column list for resources"""
    cols=list(myframe.columns)
    cols=cols[4:]
    resources_list=list(aframe['Code'])
    for index,row in cframe.iterrows():
        sindex=row['Start']
        findex=row['Finish']        
        cindex1=sindex+2
        cindex2=findex+2
        print("start={} finish={}".format(cindex1,cindex2))
        cindexs=range(cindex1,cindex2,1)
        for col in cols:
            row_index=resources_list.index(col)
            val=row[col]            
            aframe=writeInRange(aframe,row_index,cindex1,cindex2,val)           
    return aframe
    
    
    
def createCanvas(pp):
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(pp),bottomup = 0)
    return mycanvas
def getRequiredPage(start,finish):
    no_of_days=finish-start+1
    width=no_of_days*15
    if width <=842:
        paper=A4
    elif width <=1190:
        paper=A3
    elif width <=1684:
        paper=A2
    else:
        paper=A1
    return paper
def writeDayHeading(c,xlist,headinglist,y):
    c.setFont("Helvetica",8)
    c.setFillColorRGB(0,0,0.77)
    for x,txt in zip(xlist,headinglist):
        c.drawString(x,y-5,txt)
        
    
def defineUnitForGrid(start,finish,width):
    diff=finish-start
    scale=round((width/diff),2)
    return scale
def dayTextGenerator(start,finish):
    gridtext=["D"+str(i)  for i in range(start,finish+1)]
    return gridtext
def writeWorkName(c,worklist,x,ylist):
    c.setFont("Helvetica",10)
    c.setFillColorRGB(0.55,0.35,0.55)
    for y,txt in zip(ylist,worklist):
        c.drawString(x,y-5,txt)
    


def drawGrid(dframe,rframe):
    start=min(list(dframe['Start']))
    finish=max(list(dframe['Finish']))
    resourcelist=rframe['Name']
    print("max={} min={}".format(start,finish))
    xint=list(range(start,finish+1))
    yint=list(range(1,len(resourcelist)+1))
    print(xint)
    width, height = landscape(A3)
    width=width-2*inch
    height=height-2*inch
    gridwidth=width-3*inch
    scale= defineUnitForGrid(start,finish,gridwidth)
    scale=15
    print(scale)
    x0=4*inch
    y0=1.5*inch
    xlist=[x0+x*scale for x in xint]
    xlist.insert(0,1.0*inch)
    ylist=[y0+y*scale for y in yint]
    print(xlist)
    pp=getRequiredPage(start,finish)
    mycanvas=createCanvas(pp)    
    mycanvas.grid(xlist,ylist )
    """generating day text"""
    daytext=dayTextGenerator(start,finish)
    print(daytext)
    xlist2=xlist[1:-1]
    #writeDayHeading(mycanvas,xlist2,daytext,ylist[1])
    """writing work name"""
    #resourcelist=dframe['Name']
    writeWorkName(mycanvas,resourcelist,1.0*inch,ylist[2:])
    
    mycanvas.showPage()
    mycanvas.save()

In [ ]:


def createColorList():
    mylist=[colors.aqua,colors.beige,colors.cyan,colors.khaki,colors.lawngreen,
            colors.lightcoral,colors.lightskyblue]
    return mylist

In [ ]:
"""PDFgen related function"""
def createPage(out_file_name):    
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(A4),bottomup =0)
    width, height = landscape(A4)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue
def writeListVertical(mylist,c_x,c_y,mycanvas,dely):
    #dely=25
    #delx=25
    y=c_y
    x=c_x    
    mycanvas.setFont("Helvetica",8)
    mycanvas.setFillColorRGB(0,0,0.77)
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        y +=dely
def writeListInVerticalLine(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)
    for txt,y1 in zip(txtlist,ylist):
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,str(txt))
def writeListInVerticalLineAllocation(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)    
    #print(txtlist)
    #print(vallist)
    for txt,y1 in zip(txtlist,ylist):
        if txt>0:
            mystr=str(txt)
        else:
            mystr="" 
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,str(txt))
def writeListInHorizontalLine(txtlist,y,xlist,mycanvas,cellwidth,cellheight,text_align):
    yc=y+math.floor(cellheight/2)
    for txt,x1 in zip(txtlist,xlist):
        xc=x1+math.floor((1.0*cellwidth)/2) 
        mystr=str(txt)
        if text_align==1:            
            mycanvas.drawCentredString(xc,yc,mystr)
        else:
            mycanvas.drawString(xc,yc,mystr)
        
def writeListHorizontal(mylist,c_x,c_y,mycanvas,delx):
    #dely=25
    #delx=25
    y=c_y
    x=c_x
    mycanvas.setFont("Helvetica",fontsize)
    mycanvas.setFillColorRGB(0,0,0.77)    
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        x +=delx
def drawGridVertical(mylist,xlist,c_y,delta,mycanvas):
    ylist=[c_y+i*delta for i in range(len(mylist)+1)]
    mycanvas.grid(xlist,ylist)
    return ylist

def fillGridVertical(mylist,xlist,c_y,delta,mycanvas,colorlist):
    ylist1=[c_y+i*delta for i in range(len(mylist)+1)]
    ylist=ylist1[1:-2]
    color_index=[i%7 for i in range(0,len(ylist))]
    mycolors=[colorlist[i] for i in color_index]
    x=xlist[0] 
    vallist=mylist[1:-2]
    for val,y,fill_color in zip(vallist,ylist,mycolors):
        if val >0:
            mycanvas.setFillColor(fill_color)
            mycanvas.rect(x,y,delta,delta,fill=1)
        
    
def writeHeading(dframe,canvas_name):
    code=list(dframe['Code'])
    code.insert(0,"code")
    names=list(dframe['Resource_name'])
    names.insert(0,'Resource_name')
    units=list(dframe['Unit'])
    units.insert(0,"Unit")
    c_x=1.0*inch
    c_y=1.0*inch
    xlist=[1*inch,1.5*inch]
    delta=25
    fontsize=10
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77)    
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    #writeListVertical(code,c_x,c_y,canvas_name,25)
    writeListInVerticalLine(code,ylist,c_x,canvas_name,0.5*inch,25)
    c_x=1.5*inch
    xlist=[1.5*inch,3.0*inch]
    #writeListVertical(names,c_x,c_y,canvas_name,25)
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(names,ylist,c_x,canvas_name,1.5*inch,25)
    c_x=3.0*inch
    xlist=[3.0*inch,3.7*inch]
    ylist=drawGridVertical(units,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(units,ylist,c_x,canvas_name,0.7*inch,25)
    #c_x=3.0*inch
    #writeListVertical(units,c_x,c_y,canvas_name,25)
    return 3.7*inch

def writeDayHeading(start_day,finish_day,cellWidth,cellHeight,canvas_name,c_x):
    daytext=dayTextGenerator(start_day,finish_day)
    print(daytext)
    multiplier=[i%21 for i in range(start_day,finish_day+1)]
    xlist=[c_x+(i-1)*cellWidth for i in multiplier]
    c_y=1.0*inch
    print("xlist={}".format(xlist))
    print("days={}".format(daytext))
    fontsize=8
    canvas_name.setFont("Helvetica",fontsize)
    #canvas_name.setFillColorRGB(1,1,1)
    writeListInHorizontalLine(daytext,c_y,xlist,canvas_name,cellWidth,cellHeight,1)
    
def writeDailyAllocationData(dframe,canvas_name,c_x,start_day,end_day,cellWidth,cellHeight):
    daytext=dayTextGenerator(start_day,end_day)
    x1=c_x
    c_y=1.0*inch
    delta=cellHeight
    fontsize=6
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77) 
    myclolor_list=createColorList()
    print(myclolor_list)
    for day in daytext:        
        vlist=list(dframe[day])
        vlist.insert(0,"")
        x2=x1+cellWidth
        xlist=[x1,x2]
        ylist=drawGridVertical(vlist,xlist,c_y,delta,canvas_name)
        fillGridVertical(vlist,xlist,c_y,delta,canvas_name,myclolor_list)
        canvas_name.setFillColorRGB(0,0,0)
        vlist1=vlist[1:]
        ylist1=ylist[1:]
        writeListInVerticalLineAllocation(vlist1,ylist1,x1,canvas_name,cellWidth,cellHeight)
        x1+=cellWidth
        print(vlist)
        
    
 

In [ ]:
myfolder='E:\Website_26_07_2020\cmis6\Civilworks cost\Spec for EMB PROT Works'
#myfolder=r'F:\website\cmis6\Civilworks cost\Spec for EMB PROT Works'
input_path=os.path.join(myfolder,'Resources.xlsx')
filename="Rresource_uses.pdf"
output_excel_name=os.path.join(myfolder,'Resources_dframe.xlsx')    
out_file_name=os.path.join(myfolder,filename)
sheetName="Allocation"
myframe=pd.read_excel(input_path,sheet_name=sheetName)
myframe.fillna(0,inplace=True)
myframe
sheetName="Resource_List"
resoruce_frame=pd.read_excel(input_path,sheet_name=sheetName)
resoruce_frame.fillna(0,inplace=True)
resoruce_frame
"""generating allocation matrix"""
allocation_df=buildAllocationMatrix(myframe,resoruce_frame)
allocation_df=calcualteResourceAllocation(myframe,allocation_df)

In [ ]:
drawGrid(myframe,resoruce_frame)

In [ ]:
#width, height = landscape(A3)
width, height = A3
print(width)

In [ ]:
print(height)

In [ ]:
print(1*inch)

In [ ]:
allocation_df

In [ ]:
cols=list(myframe.columns)
print(cols)

In [ ]:
cols=cols[4:]
cols

In [ ]:
resources_list=list(allocation_df['Code'])

In [ ]:
resources_list

In [ ]:
myframes=[]
mynames=[]
myframes.append(allocation_df)
mynames.append("Resourses_uses")
saveDataFrame(myframes,output_excel_name,mynames)

In [ ]:
filename="test_x_5.pdf"    
testfile=os.path.join(myfolder,filename)
myvalue=createPage(testfile)

page=myvalue["canvas"]
c_x=myvalue["c_x"]
c_y=myvalue["c_y"]
"""Writing test list"""
mylist=list(allocation_df['Code'])
mylist.insert(0,"")
#writeListVertical(mylist,c_x,c_y,page,25)
c_x=writeHeading(allocation_df,page)
writeDayHeading(1,20,25,25,page,c_x)
#page.drawString("Ok",100,100)
writeDailyAllocationData(allocation_df,page,c_x,1,20,25,25)

page.showPage()
page.save()


In [ ]:
help(canvas)

In [ ]:
from reportlab.lib import colors
mycolors=colors.getAllNamedColors()
print(mycolors)